In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from DNN_module import Net
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [2]:
%run NCoinDP_functions.ipynb

In [5]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu"
device = torch.device(dev) 

2.1.2+cu121
True
NVIDIA A10


In [ ]:
# Default : cuda
torch.set_default_device('cuda')

# Number of train and validation data
Ls = [200000, 300000, 400000]
L_val  = 50000

# Define the batch size
BATCH_SIZE = 64

# Number of Epochs
N_EPOCHS = 250
    
# Number of Simulations
sims = 50

for sim in range(0, sims):
    tmp = "syn_data/OU_sim_n3000.pt"
    tmp = torch.load(tmp)

    # Data import
    [X, Y] = tmp
    X = X.to(device)
    Y = Y.to(device)

    torch.manual_seed(1000 + sim)
    
    for L in Ls:
        L_train = L
        indexes = torch.randperm(L_train + L_val)
    
        # Divide Data
        X_train = X[indexes[0:L_train]]
        Y_train = Y[indexes[0:L_train]]

        X_val = X[indexes[L_train:(L_train + L_val)]]
        Y_val = Y[indexes[L_train:(L_train + L_val)]]
    
        # Use torch.utils.data to create a DataLoader that will take care of creating batches
        dataset = TensorDataset(X_train, Y_train)
        dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, generator=torch.Generator(device=device))

        # Get the dataset size for printing (it is equal to N_SAMPLES)
        dataset_size = len(dataloader.dataset)

        # Define the input and output dimensions
        D_in, D_out = X_train.size()[1], Y_train.size()[1]


        # Create an instance of the Net class with specified dimensions
        net = Net(D_in = D_in, D_out = D_out)

        # Model name
        model_save_name = 'nets/SA2/net'+str(L)+"_"+str(sim)+'.pt'
        path = F"./{model_save_name}"

        # The nn package also contains definitions of popular loss functions; in this case we will use Mean Squared Error (MSE) as our loss function.
        loss_fn = torch.nn.MSELoss(reduction='sum')
        learning_rate = 1e-4
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

        train_error_plt = []
        val_error_plt = []

        torch.manual_seed(2000 + sim)

        # Loop over epochs
        for epoch in range(N_EPOCHS):
            for id_batch, (x_batch, y_batch) in enumerate(dataloader):
                y_batch_pred = net(x_batch)
                loss = loss_fn(y_batch_pred, y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if epoch % 50 ==0 and id_batch % 100 == 0:
                    loss, current = loss.item(), (id_batch + 1)* len(x_batch)
                    print(f"train_loss: {loss/BATCH_SIZE:>7f}  [{current:>5d}/{dataset_size:>5d}]")

            with torch.no_grad():
                   net.eval()
                   theta_pred_train = net(X_train)
                   train_loss = loss_fn(theta_pred_train,Y_train) / L_train
                   train_error_plt = np.append(train_error_plt, train_loss.to("cpu"))

                   theta_pred_val = net(X_val)
                   val_loss = loss_fn(Y_val, theta_pred_val) / L_val
                   val_error_plt = np.append(val_error_plt, val_loss.to("cpu"))

            if epoch % 10 ==0:
                print(f"Epoch {epoch + 1}\n-------------------------------")
                print(f"train_loss {train_loss:>7f} val_loss {val_loss:>7f}")
                print("learning rate: ", learning_rate, ", L: ", L, ", sim: ", sim)

            ## Choose Best Model
            if val_error_plt[epoch] == np.min(val_error_plt):
                 best=epoch
                 torch.save(net.state_dict(), path)

            if epoch % 100 ==99:
                net.load_state_dict(torch.load(path))
                learning_rate = max(learning_rate * 1e-1, 1e-9)


train_loss: 16.841385  [   64/200000]
train_loss: 6.429161  [ 6464/200000]
train_loss: 5.771806  [12864/200000]
train_loss: 3.279193  [19264/200000]
train_loss: 1.398955  [25664/200000]
train_loss: 0.800822  [32064/200000]
train_loss: 0.526072  [38464/200000]
train_loss: 0.339178  [44864/200000]
train_loss: 0.191689  [51264/200000]
train_loss: 0.211995  [57664/200000]
train_loss: 0.215071  [64064/200000]
train_loss: 0.178395  [70464/200000]
train_loss: 0.148805  [76864/200000]
train_loss: 0.131294  [83264/200000]
train_loss: 0.115533  [89664/200000]
train_loss: 0.178908  [96064/200000]
train_loss: 0.121493  [102464/200000]
train_loss: 0.167306  [108864/200000]
train_loss: 0.169280  [115264/200000]
train_loss: 0.187463  [121664/200000]
train_loss: 0.214297  [128064/200000]
train_loss: 0.149557  [134464/200000]
train_loss: 0.335281  [140864/200000]
train_loss: 0.242648  [147264/200000]
train_loss: 0.122909  [153664/200000]
train_loss: 0.150471  [160064/200000]
train_loss: 0.178709  [1664

train_loss: 0.077477  [153664/200000]
train_loss: 0.081890  [160064/200000]
train_loss: 0.126308  [166464/200000]
train_loss: 0.110903  [172864/200000]
train_loss: 0.099575  [179264/200000]
train_loss: 0.131298  [185664/200000]
train_loss: 0.089558  [192064/200000]
train_loss: 0.094008  [198464/200000]
Epoch 201
-------------------------------
train_loss 0.113841 val_loss 0.113159
learning rate:  1.0000000000000002e-06 , L:  200000 , sim:  0
Epoch 211
-------------------------------
train_loss 0.105831 val_loss 0.105295
learning rate:  1.0000000000000002e-06 , L:  200000 , sim:  0
Epoch 221
-------------------------------
train_loss 0.105615 val_loss 0.105181
learning rate:  1.0000000000000002e-06 , L:  200000 , sim:  0
Epoch 231
-------------------------------
train_loss 0.116747 val_loss 0.116503
learning rate:  1.0000000000000002e-06 , L:  200000 , sim:  0
Epoch 241
-------------------------------
train_loss 0.113391 val_loss 0.113318
learning rate:  1.0000000000000002e-06 , L:  200

train_loss: 0.144217  [   64/300000]
train_loss: 0.118400  [ 6464/300000]
train_loss: 0.142932  [12864/300000]
train_loss: 0.086186  [19264/300000]
train_loss: 0.118927  [25664/300000]
train_loss: 0.089540  [32064/300000]
train_loss: 0.122554  [38464/300000]
train_loss: 0.088770  [44864/300000]
train_loss: 0.093079  [51264/300000]
train_loss: 0.116921  [57664/300000]
train_loss: 0.087637  [64064/300000]
train_loss: 0.090423  [70464/300000]
train_loss: 0.104021  [76864/300000]
train_loss: 0.089750  [83264/300000]
train_loss: 0.067905  [89664/300000]
train_loss: 0.110376  [96064/300000]
train_loss: 0.101388  [102464/300000]
train_loss: 0.110055  [108864/300000]
train_loss: 0.094283  [115264/300000]
train_loss: 0.078386  [121664/300000]
train_loss: 0.097645  [128064/300000]
train_loss: 0.127707  [134464/300000]
train_loss: 0.123362  [140864/300000]
train_loss: 0.097724  [147264/300000]
train_loss: 0.092757  [153664/300000]
train_loss: 0.091203  [160064/300000]
train_loss: 0.135090  [16646

In [ ]:
print(path)

In [ ]:
#net.load_state_dict(torch.load("./OU_0.pt"))
net.load_state_dict(torch.load(path))

torch.set_default_device('cpu')
print(np.min(val_error_plt))
print(np.argmin(val_error_plt))
plt.plot(np.arange(N_EPOCHS), train_error_plt, color = "r")
plt.plot(np.arange(N_EPOCHS), val_error_plt)
plt.legend(["train", "validation"], loc ="upper right")

In [ ]:
net = net.to(torch.device('cpu'))